<a href="https://colab.research.google.com/github/inesbsilveira/estufas/blob/main/GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# isto deveria estar dentro da função??

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=mwslWFSOgusRBeD7vQQbdiWedGb7OirG8t1fxhJh8Z0&tc=NpCUat59KJ1JgfZkqWYgVNOi0_3BNTKaPq5l1FqgJY4&cc=Ee9myK6IR_gTjic5hqxczrLP_ST3bAqUWPOUKTWx8wc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlPamcze1_fBir-z31zPuOBhRJLMEZKfKZ57EtTJglZJRG8UVWrLcg

Successfully saved authorization token.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Function to extract images from google earth engine

"""
Inputs:
- extent = (lonmin, lonmax, latmin, latmax)
- start_date and end_date = 'YYY-MM-DD'
- band_name = 'B2', 'B3', 'B4' or 'B8'
- folder_name = 'folder_name' is where the tiff file is stored
Output: tiff file saved in folder_name
"""

def gee_images (extent, start_date, end_date, band_name, folder_name):
    # Define the extent
    lonmin, lonmax, latmin, latmax = extent

    region = ee.Geometry.Polygon(
        [[
            [lonmin, latmin],
            [lonmax, latmin],
            [lonmax, latmax],
            [lonmin, latmax]
        ]])

    # Define the Sentinel-2 image collection and filter it by date and region
    sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(region) \
        .filterDate(start_date, end_date)  # Change the date range as needed

    # Define a function to mask clouds using the QA60 band
    def maskClouds(image):
        cloudBitMask = 1 << 10
        cirrusBitMask = 1 << 11
        qa60 = image.select(['QA60'])
        cloudMask = qa60.bitwiseAnd(cloudBitMask).eq(0) \
            .And(qa60.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(cloudMask)

    # Map the maskClouds function over the image collection
    sentinel2 = sentinel2.map(maskClouds)

    # Define a function to add a band containing the date of the image
    def addDate(image):
        return image.addBands(image.metadata('system:time_start'))

    # Map the addDate function over the image collection
    sentinel2 = sentinel2.map(addDate)

    # Export each image to Google Drive
    image_list = sentinel2.toList(sentinel2.size())
    for i in range(0, sentinel2.size().getInfo()):
        image = ee.Image(image_list.get(i))
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        task = ee.batch.Export.image.toDrive(
            image=image.select([band_name]),
            #image=image.select(['B2', 'B3', 'B4', 'B8']),# Select the bands you want to export
            description='Sentinel2_' + band_name + '_' + date.getInfo(),
            folder= folder_name,
            region=region,
            scale=10  # Change the scale as needed
        )
        task.start()

In [9]:
gee_images((-8.793267, -8.7410763, 37.4473346, 37.553255), '2022-07-20', '2022-07-25', 'B2', 'Estufas')